In [3]:
from operator import itemgetter

from langchain.chat_models import ChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import ChatPromptTemplate
from langchain.schema.output_parser import StrOutputParser
from langchain.schema.runnable import RunnableLambda, RunnablePassthrough
from langchain.vectorstores import FAISS
from dotenv import load_dotenv

load_dotenv()

True

In [5]:
vectorstore = FAISS.from_texts(
    ['harrison worked at kensho'], embedding = OpenAIEmbeddings()
)

retriever = vectorstore.as_retriever()

template = """
    Answer the question based only on the following context:
    {context}

    Question: {question}
"""
prompt = ChatPromptTemplate.from_template(template)
model = ChatOpenAI()

In [6]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [7]:
chain.invoke("Where did Harrison work?")

'Harrison worked at Kensho.'

In [8]:
template = """Answer the question based only on the following context:
{context}

Question: {question}

Answer in the following language: {language}
"""

prompt = ChatPromptTemplate.from_template(template)

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
        "language": itemgetter("language")
    }
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke({"question": "Where did Harrison work?", "language": "italian"}))

Harrison ha lavorato a Kensho.


### Conversational Retrieval Chain


In [9]:
from langchain.schema import format_document
from langchain.schema.runnable import RunnableMap
from langchain.prompts.prompt import PromptTemplate

In [10]:
_template = """
Given the following conversation and a follow up question, rephrase the follow up question to be a standalone question, in its original language.

Chat History:
{chat_history}
Follow Up Input: {question}
Standalone Question:
"""
CONDENSE_QUESTION_PROMPT = PromptTemplate.from_template(_template)

In [11]:
template = """
Answer the question based only on the following context:
{context}

Question: {question}
"""
ANSWER_PROMPT = ChatPromptTemplate.from_template(template)

In [12]:
DEFAULT_DOCUMENT_PROMPT = PromptTemplate.from_template(template = "{page_content}")

def _combine_documents(
    docs, document_prompt = DEFAULT_DOCUMENT_PROMPT, document_separator = "\n\n"
):
    doc_strings = [format_document(doc, document_prompt) for doc in docs]
    return document_separator.join(doc_strings)



In [13]:
from typing import List, Tuple

def _format_chat_history(chat_history: List[Tuple]) -> str:
    buffer = ""
    for dialogue_turn in chat_history:
        print(dialogue_turn)
        human = "Human: " + dialogue_turn[0]
        ai = "Assitant: " + dialogue_turn[1]
        buffer += "\n" + "\n".join([human, ai])

    return buffer

In [39]:
_inputs = RunnableMap(
    standalone_question = RunnablePassthrough.assign(
        chat_history = lambda x: _format_chat_history(x["chat_history"])
    )
    | CONDENSE_QUESTION_PROMPT
    | ChatOpenAI(temperature = 0.7)
    | StrOutputParser(),
)

_context = {
    "context": itemgetter("standalone_question") | retriever | _combine_documents ,
    "question": lambda x: x["standalone_question"],
}

# conversation_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI() 
conversation_qa_chain = _inputs | _context | ANSWER_PROMPT | ChatOpenAI(temperature = 0.5) 

In [40]:
conversation_qa_chain.invoke(
    {
        "question": "where did harrison work?",
        "chat_history": [],
    }
).content

'Harrison was employed at Kensho.'

In [27]:
conversation_qa_chain.invoke(
    {
        "question": "where did he work?",
        "chat_history": [("Who wrote this notebook?", "Harrison")],
    }
)



('Who wrote this notebook?', 'Harrison')


{'standalone_question': 'Where did Harrison work?'}

In [75]:
# Examples FAISS.from_texts 
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.prompts import MessagesPlaceholder
from langchain.schema.runnable import RunnableLambda, RunnableConfig
from langchain.llms import OpenAI
from langchain.schema import SystemMessage

texts = ["Chinh has skill issue in Coding"]

def _documents_to_messages(docs):
    return [SystemMessage(content = doc.page_content) for doc in docs]

embeddings = OpenAIEmbeddings()
db = FAISS.from_texts(texts, embeddings)
retriever_options = db.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "Answer the question based only on the following context:"),
        MessagesPlaceholder("context"),
        ("human", "Question: {question}")
    ]
)

model = OpenAI(temperature = 1)
_documents_to_messages_runnable = RunnableLambda(_documents_to_messages)

chains = (
    {"context": retriever_options | _documents_to_messages_runnable, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

chains.invoke("What part does Chinh have issue in?")


'\n\nChinh has issues with coding.'